# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

### Tokenization

In the second cell, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Now we have the text, encode it as integers.

In [3]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [4]:
len(chars)

83

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [5]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see those same characters encoded as integers.

In [6]:
char2int

{'L': 0,
 '0': 1,
 '\n': 2,
 "'": 3,
 '!': 4,
 '6': 5,
 '.': 6,
 'U': 7,
 'Z': 8,
 ',': 9,
 'T': 10,
 'h': 11,
 'k': 12,
 'V': 13,
 '@': 14,
 '/': 15,
 'o': 16,
 'l': 17,
 '*': 18,
 ':': 19,
 '-': 20,
 'E': 21,
 '`': 22,
 '%': 23,
 'x': 24,
 'm': 25,
 '2': 26,
 '&': 27,
 '?': 28,
 'd': 29,
 'G': 30,
 'X': 31,
 'O': 32,
 'R': 33,
 'D': 34,
 'a': 35,
 ')': 36,
 'u': 37,
 'F': 38,
 'P': 39,
 'q': 40,
 'v': 41,
 '1': 42,
 'e': 43,
 'n': 44,
 'A': 45,
 'y': 46,
 '(': 47,
 'p': 48,
 'f': 49,
 '7': 50,
 's': 51,
 'z': 52,
 'J': 53,
 'I': 54,
 '3': 55,
 'K': 56,
 'H': 57,
 'C': 58,
 ' ': 59,
 '4': 60,
 '5': 61,
 '_': 62,
 'W': 63,
 'j': 64,
 'i': 65,
 'B': 66,
 '8': 67,
 'g': 68,
 'w': 69,
 't': 70,
 'M': 71,
 '"': 72,
 ';': 73,
 'Q': 74,
 'N': 75,
 'r': 76,
 'Y': 77,
 'b': 78,
 'c': 79,
 '9': 80,
 'S': 81,
 '$': 82}

In [7]:
encoded[:100]

array([58, 11, 35, 48, 70, 43, 76, 59, 42,  2,  2,  2, 57, 35, 48, 48, 46,
       59, 49, 35, 25, 65, 17, 65, 43, 51, 59, 35, 76, 43, 59, 35, 17, 17,
       59, 35, 17, 65, 12, 43, 73, 59, 43, 41, 43, 76, 46, 59, 37, 44, 11,
       35, 48, 48, 46, 59, 49, 35, 25, 65, 17, 46, 59, 65, 51, 59, 37, 44,
       11, 35, 48, 48, 46, 59, 65, 44, 59, 65, 70, 51, 59, 16, 69, 44,  2,
       69, 35, 46,  6,  2,  2, 21, 41, 43, 76, 46, 70, 11, 65, 44])

In [8]:
len(encoded)

1985223

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an intgere (via our created dictionary) and *then* converted into a column vector where only it's corresponsing integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [9]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `n_seqs` (also refered to as "batch size" in other places). Each of those sequences will be `n_steps` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ sequences. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [10]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    # Get the number of characters per batch
    batch_size = n_seqs * n_steps
    #print(f"batch_size = {batch_size}")
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr)//batch_size
    #print(f"n_batches = {n_batches}")
    #print(f"len(arr) = {len(arr)}")
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[0:n_batches*batch_size]
    #print(f"n_batches*batch_size = {n_batches*batch_size}")
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape(n_seqs, -1)
    
    ## TODO: Make batches
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        y = np.zeros(x.shape, dtype=int)
        
        # The targets, shifted by one
        if n+n_steps > arr.shape[1]-1:
            y[:, :-1], y[:, -1] = x[:, 1:n_steps], arr[:, 0]
        else:
            y[:, :-1], y[:, -1] = x[:, 1:n_steps], arr[:, n+n_steps]
                
        #y = arr[:, n+1:n+n_steps+1]
        yield x, y


### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 10 and 50 sequence steps.

In [11]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print(f"np.shape(x) = {np.shape(x)}")
print(f"np.shape(y) = {np.shape(y)}")

np.shape(x) = (10, 50)
np.shape(y) = (10, 50)


In [12]:
print('x\n', x[:, 10:20])
print('\ny\n', y[:, 10:20])

x
 [[ 2  2 57 35 48 48 46 59 49 35]
 [65 44 68 59 70 16 59 51 70 35]
 [ 9 59 65 70  3 51 59 51 43 70]
 [65 51 59 79 16 44 41 43 76 51]
 [76  4 72 59 51 35 65 29 59 70]
 [17 46 59 69 11 43 44 59 70 11]
 [16 76 59 25 43  9 72 59 51 11]
 [51 11 43 59 69 16 37 17 29 59]
 [11 43 46  3 76 43 59 48 76 16]
 [43 76 51 43 17 49  9 59 35 44]]

y
 [[ 2 57 35 48 48 46 59 49 35 25]
 [44 68 59 70 16 59 51 70 35 46]
 [59 65 70  3 51 59 51 43 70 70]
 [51 59 79 16 44 41 43 76 51 35]
 [ 4 72 59 51 35 65 29 59 70 11]
 [46 59 69 11 43 44 59 70 11 43]
 [76 59 25 43  9 72 59 51 11 43]
 [11 43 59 69 16 37 17 29 59 76]
 [43 46  3 76 43 59 48 76 16 48]
 [76 51 43 17 49  9 59 35 44 29]]


In [13]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[58 11 35 48 70 43 76 59 42  2]
 [59 35 25 59 44 16 70 59 68 16]
 [41 65 44  6  2  2 72 77 43 51]
 [44 59 29 37 76 65 44 68 59 11]
 [59 65 70 59 65 51  9 59 51 65]
 [59 54 70 59 69 35 51  2 16 44]
 [11 43 44 59 79 16 25 43 59 49]
 [73 59 78 37 70 59 44 16 69 59]
 [70 59 65 51 44  3 70  6 59 10]
 [59 51 35 65 29 59 70 16 59 11]]

y
 [[11 35 48 70 43 76 59 42  2  2]
 [35 25 59 44 16 70 59 68 16 65]
 [65 44  6  2  2 72 77 43 51  9]
 [59 29 37 76 65 44 68 59 11 65]
 [65 70 59 65 51  9 59 51 65 76]
 [54 70 59 69 35 51  2 16 44 17]
 [43 44 59 79 16 25 43 59 49 16]
 [59 78 37 70 59 44 16 69 59 51]
 [59 65 51 44  3 70  6 59 10 11]
 [51 35 65 29 59 70 16 59 11 43]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic LSTM cell as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial cell state of all zeros. This is done like so

```python
self.init_weights()
```

In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM, self.lstm
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=n_hidden, num_layers=n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer, self.dropout
        self.dropout = nn.Dropout(p=drop_prob)
        
        ## TODO: define the final, fully-connected output layer, self.fc
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # initialize the weights
        self.init_weights()
      
    
    def forward(self, x, hc):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        #print(f"input> x.size() = {x.size()}")
        ## TODO: Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        #print(f"after lstm > x.size() = {x.size()}")
        #print(f"after lstm > h.size() = {h.size()}")
        ## TODO: pass x through a droupout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        #print(f"x.size() = {x.size()}")
        #print(f"self.n_hidden = {self.n_hidden}")
        x = x.reshape(x.size()[0]*x.size()[1], self.n_hidden)
        #print(f"===> x.size() = {x.size()}")
        
        ## TODO: put x through the fully-connected layer
        x = self.fc(x)
        
        # return x and the hidden state (h, c)
        #print(f"last fc > x.size() = {x.size()}")
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        #print(f"before one_hot_encode, x.shape = {x.shape}")
        x = one_hot_encode(x, len(self.chars))
        #print(f"after one_hot_encode, x.shape = {x.shape}")
        inputs = torch.from_numpy(x)
        if cuda:
            inputs = inputs.cuda()
        
        #h = tuple([each.detach() for each in h])
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())
        

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

In [15]:
## ---- keep notebook from crashing during training --- ##
import os
import requests
import time

def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10, response=None):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    old_time = time.time()

    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        for x, y in get_batches(data, n_seqs, n_steps):
            
            if time.time() - old_time > 60:
                old_time = time.time()
                #requests.request("POST", 
                #                 "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                #                 headers={'Authorization': "STAR " + response.text})
                #KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
                #headers = {'Authorization': "STAR " + response.text}
                #requests.request("POST", KEEPALIVE_URL, headers=headers)
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            #print(f"np.shape(x) = {np.shape(x)}")
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            #print(f"np.shape(output) = {np.shape(output)}")
            loss = criterion(output, targets.view(n_seqs*n_steps))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(n_seqs*n_steps))
                    #print(f"val_loss = {val_loss}")
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Time to train

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes (number of sequences and number of steps), and start the training. With the train function, we can set the number of epochs, the learning rate, and other parameters. Also, we can run the training on a GPU by setting `cuda=True`.

In [66]:
if 'net' in locals():
    del net

In [67]:
# define and print the net
#net = CharRNN(chars, n_hidden=512, n_layers=2)
net = CharRNN(chars, n_hidden=512, n_layers=3, drop_prob=0.5)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [68]:
n_seqs, n_steps = 128, 100
#n_seqs, n_steps = 256, 100
#n_seqs, n_steps = 128, 50
#n_seqs, n_steps = 128, 200
#n_seqs, n_steps = 128, 500

# you may change cuda to True if you plan on using a GPU!
# also, if you do, please INCREASE the epochs to 25

# Open the training log file.
log_file = 'training_log.txt'
f = open(log_file, 'w')

TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}

#response = requests.request("GET", 
#                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})
response = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS)

# TRAIN
train(net, encoded, epochs=60, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=10, response=response)

# Close the training log file.
f.close()


Epoch: 1/60... Step: 10... Loss: 3.3710... Val Loss: 3.3314
Epoch: 1/60... Step: 20... Loss: 3.2710... Val Loss: 3.2864
Epoch: 1/60... Step: 30... Loss: 3.2485... Val Loss: 3.2477
Epoch: 1/60... Step: 40... Loss: 3.1843... Val Loss: 3.1989
Epoch: 1/60... Step: 50... Loss: 3.0934... Val Loss: 3.0778
Epoch: 1/60... Step: 60... Loss: 2.9781... Val Loss: 2.9870
Epoch: 1/60... Step: 70... Loss: 2.8606... Val Loss: 2.8699
Epoch: 1/60... Step: 80... Loss: 2.7081... Val Loss: 2.7254
Epoch: 1/60... Step: 90... Loss: 2.6321... Val Loss: 2.6221
Epoch: 1/60... Step: 100... Loss: 2.5463... Val Loss: 2.5534
Epoch: 1/60... Step: 110... Loss: 2.4888... Val Loss: 2.5093
Epoch: 1/60... Step: 120... Loss: 2.4109... Val Loss: 2.4632
Epoch: 1/60... Step: 130... Loss: 2.4180... Val Loss: 2.4230
Epoch: 2/60... Step: 140... Loss: 2.3563... Val Loss: 2.3875
Epoch: 2/60... Step: 150... Loss: 2.3290... Val Loss: 2.3528
Epoch: 2/60... Step: 160... Loss: 2.3076... Val Loss: 2.3281
Epoch: 2/60... Step: 170... Loss:

Epoch: 10/60... Step: 1350... Loss: 1.2797... Val Loss: 1.4527
Epoch: 10/60... Step: 1360... Loss: 1.2807... Val Loss: 1.4502
Epoch: 10/60... Step: 1370... Loss: 1.2878... Val Loss: 1.4497
Epoch: 10/60... Step: 1380... Loss: 1.2989... Val Loss: 1.4515
Epoch: 10/60... Step: 1390... Loss: 1.3316... Val Loss: 1.4463
Epoch: 11/60... Step: 1400... Loss: 1.3300... Val Loss: 1.4417
Epoch: 11/60... Step: 1410... Loss: 1.3294... Val Loss: 1.4336
Epoch: 11/60... Step: 1420... Loss: 1.3280... Val Loss: 1.4346
Epoch: 11/60... Step: 1430... Loss: 1.2883... Val Loss: 1.4328
Epoch: 11/60... Step: 1440... Loss: 1.3125... Val Loss: 1.4349
Epoch: 11/60... Step: 1450... Loss: 1.2455... Val Loss: 1.4332
Epoch: 11/60... Step: 1460... Loss: 1.2703... Val Loss: 1.4335
Epoch: 11/60... Step: 1470... Loss: 1.2609... Val Loss: 1.4370
Epoch: 11/60... Step: 1480... Loss: 1.2862... Val Loss: 1.4365
Epoch: 11/60... Step: 1490... Loss: 1.2712... Val Loss: 1.4375
Epoch: 11/60... Step: 1500... Loss: 1.2686... Val Loss:

Epoch: 20/60... Step: 2660... Loss: 1.1874... Val Loss: 1.3677
Epoch: 20/60... Step: 2670... Loss: 1.1822... Val Loss: 1.3734
Epoch: 20/60... Step: 2680... Loss: 1.1813... Val Loss: 1.3764
Epoch: 20/60... Step: 2690... Loss: 1.1675... Val Loss: 1.3687
Epoch: 20/60... Step: 2700... Loss: 1.1788... Val Loss: 1.3735
Epoch: 20/60... Step: 2710... Loss: 1.1554... Val Loss: 1.3735
Epoch: 20/60... Step: 2720... Loss: 1.1530... Val Loss: 1.3775
Epoch: 20/60... Step: 2730... Loss: 1.1510... Val Loss: 1.3782
Epoch: 20/60... Step: 2740... Loss: 1.1558... Val Loss: 1.3750
Epoch: 20/60... Step: 2750... Loss: 1.1685... Val Loss: 1.3746
Epoch: 20/60... Step: 2760... Loss: 1.1445... Val Loss: 1.3786
Epoch: 20/60... Step: 2770... Loss: 1.1879... Val Loss: 1.3713
Epoch: 20/60... Step: 2780... Loss: 1.2188... Val Loss: 1.3734
Epoch: 21/60... Step: 2790... Loss: 1.1879... Val Loss: 1.3689
Epoch: 21/60... Step: 2800... Loss: 1.2059... Val Loss: 1.3655
Epoch: 21/60... Step: 2810... Loss: 1.2051... Val Loss:

Epoch: 29/60... Step: 3970... Loss: 1.1386... Val Loss: 1.3602
Epoch: 29/60... Step: 3980... Loss: 1.0985... Val Loss: 1.3582
Epoch: 29/60... Step: 3990... Loss: 1.1130... Val Loss: 1.3611
Epoch: 29/60... Step: 4000... Loss: 1.1258... Val Loss: 1.3562
Epoch: 29/60... Step: 4010... Loss: 1.1024... Val Loss: 1.3574
Epoch: 29/60... Step: 4020... Loss: 1.1046... Val Loss: 1.3599
Epoch: 29/60... Step: 4030... Loss: 1.1200... Val Loss: 1.3534
Epoch: 30/60... Step: 4040... Loss: 1.1183... Val Loss: 1.3534
Epoch: 30/60... Step: 4050... Loss: 1.1265... Val Loss: 1.3564
Epoch: 30/60... Step: 4060... Loss: 1.1236... Val Loss: 1.3545
Epoch: 30/60... Step: 4070... Loss: 1.1143... Val Loss: 1.3629
Epoch: 30/60... Step: 4080... Loss: 1.1126... Val Loss: 1.3534
Epoch: 30/60... Step: 4090... Loss: 1.1209... Val Loss: 1.3569
Epoch: 30/60... Step: 4100... Loss: 1.0923... Val Loss: 1.3514
Epoch: 30/60... Step: 4110... Loss: 1.0934... Val Loss: 1.3580
Epoch: 30/60... Step: 4120... Loss: 1.0920... Val Loss:

Epoch: 38/60... Step: 5280... Loss: 1.0688... Val Loss: 1.3496
Epoch: 39/60... Step: 5290... Loss: 1.0686... Val Loss: 1.3497
Epoch: 39/60... Step: 5300... Loss: 1.0944... Val Loss: 1.3549
Epoch: 39/60... Step: 5310... Loss: 1.0939... Val Loss: 1.3549
Epoch: 39/60... Step: 5320... Loss: 1.0881... Val Loss: 1.3579
Epoch: 39/60... Step: 5330... Loss: 1.0672... Val Loss: 1.3556
Epoch: 39/60... Step: 5340... Loss: 1.0853... Val Loss: 1.3574
Epoch: 39/60... Step: 5350... Loss: 1.0700... Val Loss: 1.3534
Epoch: 39/60... Step: 5360... Loss: 1.0847... Val Loss: 1.3544
Epoch: 39/60... Step: 5370... Loss: 1.0730... Val Loss: 1.3630
Epoch: 39/60... Step: 5380... Loss: 1.0595... Val Loss: 1.3522
Epoch: 39/60... Step: 5390... Loss: 1.0721... Val Loss: 1.3475
Epoch: 39/60... Step: 5400... Loss: 1.0530... Val Loss: 1.3529
Epoch: 39/60... Step: 5410... Loss: 1.0606... Val Loss: 1.3589
Epoch: 39/60... Step: 5420... Loss: 1.0737... Val Loss: 1.3504
Epoch: 40/60... Step: 5430... Loss: 1.0755... Val Loss:

Epoch: 48/60... Step: 6590... Loss: 1.0323... Val Loss: 1.3461
Epoch: 48/60... Step: 6600... Loss: 1.0372... Val Loss: 1.3491
Epoch: 48/60... Step: 6610... Loss: 1.0202... Val Loss: 1.3485
Epoch: 48/60... Step: 6620... Loss: 1.0265... Val Loss: 1.3559
Epoch: 48/60... Step: 6630... Loss: 1.0429... Val Loss: 1.3409
Epoch: 48/60... Step: 6640... Loss: 1.0387... Val Loss: 1.3517
Epoch: 48/60... Step: 6650... Loss: 1.0272... Val Loss: 1.3526
Epoch: 48/60... Step: 6660... Loss: 1.0158... Val Loss: 1.3474
Epoch: 48/60... Step: 6670... Loss: 1.0197... Val Loss: 1.3435
Epoch: 49/60... Step: 6680... Loss: 1.0214... Val Loss: 1.3470
Epoch: 49/60... Step: 6690... Loss: 1.0462... Val Loss: 1.3501
Epoch: 49/60... Step: 6700... Loss: 1.0471... Val Loss: 1.3447
Epoch: 49/60... Step: 6710... Loss: 1.0470... Val Loss: 1.3483
Epoch: 49/60... Step: 6720... Loss: 1.0152... Val Loss: 1.3501
Epoch: 49/60... Step: 6730... Loss: 1.0356... Val Loss: 1.3493
Epoch: 49/60... Step: 6740... Loss: 1.0249... Val Loss:

Epoch: 57/60... Step: 7900... Loss: 1.0200... Val Loss: 1.3543
Epoch: 57/60... Step: 7910... Loss: 1.0079... Val Loss: 1.3582
Epoch: 57/60... Step: 7920... Loss: 1.0212... Val Loss: 1.3543
Epoch: 58/60... Step: 7930... Loss: 0.9965... Val Loss: 1.3451
Epoch: 58/60... Step: 7940... Loss: 1.0081... Val Loss: 1.3358
Epoch: 58/60... Step: 7950... Loss: 1.0044... Val Loss: 1.3568
Epoch: 58/60... Step: 7960... Loss: 1.0229... Val Loss: 1.3462
Epoch: 58/60... Step: 7970... Loss: 1.0187... Val Loss: 1.3488
Epoch: 58/60... Step: 7980... Loss: 1.0036... Val Loss: 1.3584
Epoch: 58/60... Step: 7990... Loss: 1.0170... Val Loss: 1.3509
Epoch: 58/60... Step: 8000... Loss: 0.9944... Val Loss: 1.3539
Epoch: 58/60... Step: 8010... Loss: 0.9884... Val Loss: 1.3549
Epoch: 58/60... Step: 8020... Loss: 1.0194... Val Loss: 1.3588
Epoch: 58/60... Step: 8030... Loss: 1.0024... Val Loss: 1.3583
Epoch: 58/60... Step: 8040... Loss: 0.9997... Val Loss: 1.3551
Epoch: 58/60... Step: 8050... Loss: 0.9947... Val Loss:

In [69]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(pytorch_total_params)

5467731


In [70]:
for name, param in net.named_parameters():
    print(param.numel())
    if param.requires_grad:
        print(name, param.data, np.shape(param.data))

169984
lstm.weight_ih_l0 tensor([[-0.1041,  0.3256,  0.0891,  ...,  0.1243, -0.3066,  0.0149],
        [ 0.1382,  0.4921, -0.1189,  ...,  0.1123,  0.1239,  0.0088],
        [ 0.4535,  0.0920, -0.1406,  ...,  0.0013, -0.1480,  0.0351],
        ...,
        [-0.4813,  0.0155,  0.6160,  ..., -0.1107,  0.2155,  0.0186],
        [ 0.1947,  0.0007, -0.1506,  ...,  0.1012,  0.3050,  0.0062],
        [ 0.1426,  0.2602,  0.0870,  ...,  0.0231,  0.1102, -0.0302]],
       device='cuda:0') torch.Size([2048, 83])
1048576
lstm.weight_hh_l0 tensor([[-0.2887,  0.1994, -0.0559,  ..., -0.2220, -0.3536, -0.1042],
        [ 0.0780, -0.1968, -0.1950,  ..., -0.0323, -0.2366,  0.2558],
        [-0.2289,  0.2406, -0.1880,  ..., -0.2946,  0.0039, -0.0981],
        ...,
        [-0.2898,  0.0888, -0.0320,  ..., -0.3030, -0.2033,  0.3533],
        [ 0.3993,  0.2884, -0.0980,  ..., -0.2584,  0.1882, -0.2257],
        [-0.0027, -0.1724, -0.1279,  ..., -0.2727,  0.3080, -0.2302]],
       device='cuda:0') torch.Size

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `n_seqs` - Number of sequences running through the network in one pass.
* `n_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [71]:
# change the name, for saving multiple files
model_name = 'rnn_1_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

Now that the model is trained, we'll want to sample from it. To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### Top K sampling

Our predictions come from a categorcial probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [72]:
def sample(net, size, prime='The', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [73]:
print(sample(net, 2000, prime='Anna', top_k=5, cuda=True))

Anna and
Levin. She was not long before the capital is a sense of delight,
then the silence was a little sinking and work on a chair. He saw
his wife would have said, and that a long while she had a little ashamed,
and how he was always to see him. But he would say nothing of
have a desire to go to see him, that in this stretch of a man would all
sort of a part to stop you," said Stepan Arkadyevitch with a
plain in the same time.

All the peasants had all been seaned at the part, and the master of
the soul of what she was to be angry, but to see his brother Nikolay at
her mother. They had thought of surmanes and charm, but as it he
wanted, a silence, a more chambagne, walked into the sound.

"Wait a minute!" she said, looking inquiringly at Levin that he
had not time to drive out of the door at the choid, and she
was always those settlement on a successful properly which he stopped a
love for the strange to have been alone in two months. Stepan
Arkadyevitch was thrown into the wet, the

## Loading a checkpoint

In [74]:
# Here we have loaded in a model that trained over 1 epoch `rnn_1_epoch.net`
with open('rnn_1_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [75]:
# Change cuda to True if you are using GPU!
print(sample(loaded, 2000, cuda=True, top_k=5, prime="And Levin said"))

And Levin said
he, that this wonders, the place in the
soul of his case, and then seemed to show his back from time to do
this, had no sort of commiteration in his heart, and there was
a little case on a muscal of terror, he sat down beside his work,
he heard all the solition of her face, without settling it, she
could not help being said. And at this moment he saw her. He
had been caught, to bore the peace at the prince where they had time
to great this as it was, she was asked. These might all the
steps was sometimes breadth over her. His mother-seered him,
because his wife..." And some ready fellows were now as he said
themselves, and how to stand it approached him about the work, but
it was that the more and more than anyone would arise things.
Anna sat straight by the corridor. She could not try to go in.

"Why should you be as a servant for?"

"No, I don't understand."

"Oh," said Alexey Alexandrovitch, with smile and timid smile.

"Will you be thrown up for?"

"No, I can't stay.